In [1772]:
import pandas as pd
import numpy as np
import os

# Data wrangling

## Importing Data

### Auditors file

In [1773]:
path="/Users/atousachaharsoughshirazi/Engage3"
os.chdir(path)

In [1774]:
file_auditors="auditors.csv"
df_auditors=pd.read_csv(file_auditors,header=0)

In [1775]:
df_auditors

,Auditor ID,First,Last,Region
0,234,Sue,Smith,Northern California
1,536,Bob,Smith,Northern California
2,98,Jack,Smith,New York
3,203,Jill,Smith,New York
4,304,Jerry,Johnson,Texas
5,63,Randy,Johnson,Texas
6,1326,Mike,Johnson,Kansas
7,713,Dave,Johnson,Kansas


In [1776]:
df_auditors.shape

(8, 4)

In [1777]:
df_auditors.nunique()

Auditor ID    8
First         8
Last          2
Region        4
dtype: int64

 **Notes for the auditors file:**

 - There are 4 regions and 8 auditors in auditors file. 
 - Hawaii is missing. So there is no auditor recorded for Hawaii stores.
 - There are no null values.

### Prices file

In [1778]:
file_prices="prices.csv"
df_prices=pd.read_csv(file_prices,header=0)

In [1779]:
df_prices.head()

,Auditor ID,Date,Price,Store ID,UPC
0,234,10/18/17,24.95,66999,268588472
1,234,10/27/17,49.71,66999,475245085
2,234,10/20/17,25.75,66999,126967843
3,234,10/23/17,18.81,66999,708930835
4,234,10/23/17,33.32,66999,325885139


In [1780]:
df_prices.shape

(12315, 5)

In [1781]:
df_prices.nunique()

Auditor ID       8
Date            14
Price         2596
Store ID        20
UPC           1000
dtype: int64

In [1782]:
df_prices['Auditor ID'].value_counts()

304     2288
98      2090
1326    1785
203     1744
63      1477
713     1302
536     1217
234      412
Name: Auditor ID, dtype: int64

**Notes for the prices file:**

- There are 1000 unique UPC codes.
- 20 unique stores
- 12315 rows in the prices file.
- No null values

### Stores file

In [1783]:
df_stores=pd.read_json (r'/Users/atousachaharsoughshirazi/Engage3/stores.json')
df_stores.head()

,Store ID,Banner,Region
0,66999,Walmart,Northern California
1,4698,Trader Joes,Northern California
2,39482,Safeway,Northern California
3,34957,Whole Foods,Northern California
4,12837,Walmart,New York


In [1784]:
df_stores.shape

(28, 3)

In [1785]:
df_stores.nunique()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/displayhook.py:275: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


Store ID    28
Banner       6
Region       5
dtype: int64

In [1786]:
storeid_st=list(df_stores['Store ID'].values)
storeid_pr=list(df_prices['Store ID'].values)

In [1787]:
missing_id=[]
for id in storeid_pr:
    if id not in storeid_st:
        if id not in missing_id:
            missing_id.append(id)
print (missing_id)

[60957, 38472]


In [1788]:
df_prices[df_prices['Store ID'].isin(missing_id)]['Store ID'].value_counts()

38472    503
60957    301
Name: Store ID, dtype: int64

**Notes for the stores file:**

- 28 store in the stores file. 
- There are 8 stores that do not appear in the prices file. So these stores were not audited.
- There are two stores in the price file that are missing from the stores file. That means we don't have the banner and region information for these stores. The missing region information can be filled with the region information in the auditors file.
- 804 rows correspond to these two stores.
- No null values

## Merging files

In this section we will merge the prices and stores table on "Store ID'. Since, there are more store ids in the stores file compared to prices file (28 compared to 20), we will do an outer join so we don't miss any data. Then merge the result with audiotors file on "Auditor ID".

In [1789]:
merged_pr_st = pd.merge( df_prices, df_stores, how='outer', on="Store ID")

In [1790]:
merged_pr_st.shape

(12325, 7)

In [1791]:
result_final = pd.merge(merged_pr_st, df_auditors, how='outer', on="Auditor ID")

In [1792]:
result_final.head()

,Auditor ID,Date,Price,Store ID,UPC,Banner,Region_x,First,Last,Region_y
0,234.0,10/18/17,24.95,66999,268588472.0,Walmart,Northern California,Sue,Smith,Northern California
1,234.0,10/27/17,49.71,66999,475245085.0,Walmart,Northern California,Sue,Smith,Northern California
2,234.0,10/20/17,25.75,66999,126967843.0,Walmart,Northern California,Sue,Smith,Northern California
3,234.0,10/23/17,18.81,66999,708930835.0,Walmart,Northern California,Sue,Smith,Northern California
4,234.0,10/23/17,33.32,66999,325885139.0,Walmart,Northern California,Sue,Smith,Northern California


In [1793]:
result_final.shape

(12325, 10)

In [1794]:
result_final.isnull().sum()

Auditor ID     10
Date           10
Price          10
Store ID        0
UPC            10
Banner        804
Region_x      804
First          10
Last           10
Region_y       10
dtype: int64

There are 10 rows which have a null value for UPC. These rows are dropped since these rows are not assigned to a particular product and don't give us useful information.

In [1795]:
result_final=result_final[result_final['UPC'].notna()]
result_final.shape

(12315, 10)

In [1796]:
result_final.isnull().sum()

Auditor ID      0
Date            0
Price           0
Store ID        0
UPC             0
Banner        804
Region_x      804
First           0
Last            0
Region_y        0
dtype: int64

- There are 804 missing values in the banner and region(Region_x) columns in the prices file. These values can be filled with the region values in the second region column. 
- **Discrepancy:** As it is shown by the analysis below, there are 306 rows in the table with auditor Jack Smith with id=98. This auditor is asigned to New york region according to auditors table. But the store associated with these rows has store id=39482 and is located in Northern California. There must be a mistake here. Either the auditor id is not recorded properly or the store id is not correct. In order to fix this problem we need to know more details about the audit process. For the purpose of this assignment I assume that the region given in the stores table is correct and drop the region_x column.


In [1797]:
re=result_final['Region_x'].compare(result_final['Region_y'], keep_equal=False)

In [1798]:
re[re.self=="Northern California"].other.value_counts()

New York    306
Name: other, dtype: int64

In [1799]:
result_final[(result_final['Region_x']=='Northern California')&(result_final['Region_y']=='New York')&(result_final['Auditor ID']==98)]['Store ID'].value_counts()

39482    306
Name: Store ID, dtype: int64

In [1800]:
df_prices[df_prices['Store ID']==39482]['Auditor ID'].value_counts()

98    306
Name: Auditor ID, dtype: int64

In [1801]:
re.self.value_counts()

Northern California    306
Name: self, dtype: int64

In [1802]:
re.other.value_counts()

New York               809
Northern California    301
Name: other, dtype: int64

In [1803]:
re.isnull().sum()

self     804
other      0
dtype: int64

Since the 

In [1804]:
result_final.drop(columns=['Region_x', 'Store ID','Date', 'Auditor ID', 'First','Last'], inplace=True)

In [1805]:
result_final.rename(columns={"Region_y": "Region"}, inplace=True)

In [1806]:
result_final.shape

(12315, 4)

## Results and analysis

### Table with missing banner info

The table below shows result of a cross table including the data with missing banner.

In [1807]:
table_missing_banner = pd.pivot(data=result_final,index=['Banner','UPC'], columns=['Region'])
table_missing_banner

Price                                    
Region                  Kansas New York Northern California  Texas
Banner      UPC                                                   
NaN         11873171.0     NaN     5.39                5.89    NaN
            16482322.0     NaN    18.59               20.39    NaN
            16729338.0     NaN     8.29                9.09    NaN
            16829288.0     NaN     3.69                3.99    NaN
            16999755.0     NaN      NaN                5.99    NaN
...                        ...      ...                 ...    ...
Whole Foods 995798889.0   1.99    62.39               70.39  60.59
            996262978.0   1.99    14.39               16.19  13.89
            996849471.0   1.99    12.79                 NaN    NaN
            998831540.0   1.99    39.99                 NaN  38.79
            999185078.0   1.99    58.09               65.49    NaN

[5594 rows x 4 columns]

### Table with no missing banner info

The table below shows result of a cross table excluding the data with missing banner.

In [1808]:
table_banner = pd.pivot_table(data=result_final,index=['Banner','UPC'], columns=['Region'])
table_banner

Price                                    
Region                  Kansas New York Northern California  Texas
Banner      UPC                                                   
Safeway     11873171.0     NaN     6.09                 NaN   5.19
            15052612.0   53.99      NaN                 NaN  54.49
            16482322.0   17.89      NaN                 NaN  18.09
            16729338.0    7.99     9.39                 NaN   8.09
            16829288.0    3.59     4.19                 NaN   3.59
...                        ...      ...                 ...    ...
Whole Foods 995798889.0   1.99    62.39               70.39  60.59
            996262978.0   1.99    14.39               16.19  13.89
            996849471.0   1.99    12.79                 NaN    NaN
            998831540.0   1.99    39.99                 NaN  38.79
            999185078.0   1.99    58.09               65.49    NaN

[4931 rows x 4 columns]

### Table with sorted UPC values

The table below shows result of a cross table excluding the data with missing banner and sorted by ascending UPC.

In [1809]:
df = table.sort_values(by=['Banner'], ascending=True)
df.head()

Price                                    
Region              Kansas New York Northern California  Texas
Banner  UPC                                                   
Safeway 11873171.0     NaN     6.09                 NaN   5.19
        676274385.0    NaN    48.79                 NaN  41.99
        676939567.0  38.39      NaN                 NaN    NaN
        677674656.0  19.59    22.99                 NaN  19.79
        678530217.0    NaN      NaN                 NaN  39.69

### Analysis

**Anomaly detection:**

- There are two stores in the price file that are missing from the stores file. That means we don't have the banner and region information for these stores. The missing region information can be filled with the region information in the auditors file.
- 804 rows correspond to these two stores.
- There are 10 rows which have a null value for UPC. These rows are dropped since these rows are not assigned to a particular product and don't give us useful information.
- There are 804 missing values in the banner and region(Region_x) columns in the prices file. These values can be filled with the region values in the second region column. 
- **Anomaly:** There are 306 rows in the table with auditor Jack Smith with id=98. This auditor is asigned to New york region according to auditors table. But the store associated with these rows has store id=39482 and is located in Northern California. There must be a mistake here. Either the auditor id is not recorded properly or the store id is not correct. In order to fix this problem we need to know more details about the audit process. For the purpose of this assignment I assume that the region given in the stores table is correct and drop the region_x column.